In [1]:
#from highcharts import Highchart

import pandas as pd
import datetime
import numpy as np

import random

import urllib
from sqlalchemy import create_engine

import IPython

from highcharts import Highstock
from highcharts.highstock.highstock_helper import jsonp_loader

from yzhang import ta


params = urllib.parse.quote("DRIVER={ODBC Driver 17 For SQL Server};SERVER=DESKTOP-DPKTO85\SQLEXPRESS;DATABASE=tzhang;TRUSTED_CONNECTION=yes;UseFMTOnly=yes;ColumnEncryption=Enabled")
ENGINE = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params, fast_executemany=True)

i_ticker_list = pd.read_sql("select distinct ticker  FROM [tzhang].[dbo].[us_mkt_data_quandl]",con = ENGINE)






In [2]:
###
### Params
###

import numpy as np
from zigzag import *

MIN_LEN = 1000

Min_sensitivity = 2
Max_sensitivity = 2

o_data63 = np.empty((0,63*3+1))
o_data126 = np.empty((0,126*3+1))
o_data189 = np.empty((0,189*3+1))

In [9]:


# Query random data

i_selected_ticker = i_ticker_list['ticker'][random.randrange(0,len(i_ticker_list))]
i_data = pd.read_sql("select *  FROM [tzhang].[dbo].[us_mkt_data_quandl] where ticker = '"+i_selected_ticker+"' order by date asc",con = ENGINE)


if len(i_data)>=MIN_LEN:
    i_data = i_data
    
else:
    raise Exception('Not enough data.')

# standardize data

i_data_norm = pd.DataFrame()
i_data_norm['date']= ((i_data['date'] - datetime.date(1970,1,1)).apply(lambda x: x.days)*24*3600*1000).values

i_data_norm['o'] = (i_data['adj_open'].values)
i_data_norm['h'] = (i_data['adj_high'].values)
i_data_norm['l'] = (i_data['adj_low'].values)
i_data_norm['c'] = (i_data['adj_close'].values)

# zigzag
h_pivots = peak_valley_pivots(i_data_norm['h'].values, 0.05, -0.05)
l_pivots = peak_valley_pivots(i_data_norm['l'].values, 0.05, -0.05)

t_pivots = h_pivots + l_pivots

i_data_norm['extrema'] = 0
i_data_norm.loc[np.where(t_pivots>0)[0] ,'extrema'] =i_data_norm.loc[np.where(t_pivots>0)[0] ,'h']
i_data_norm.loc[np.where(t_pivots<0)[0] ,'extrema'] =i_data_norm.loc[np.where(t_pivots<0)[0] ,'l']

t_data_pivot = i_data_norm.loc[np.where(t_pivots!=0)[0],['date','extrema']]



pivot_ts, pivot_price, _ = ta.zigzag(i_data_norm['date'].values, i_data_norm['h'].values,i_data_norm['l'].values,i_data_norm['c'].values, 
                                     window = 50, how = 'dynamic', percent_std = 2)

pivot_o = np.array([pivot_ts, pivot_price]).T.tolist()


# visualize data
H = Highstock()

o_data = i_data_norm.values.tolist()

H.add_data_set(o_data, 'candlestick', 'Price', yAxis = 0)
H.add_data_set(pivot_o, 'line', 'Zigzag', yAxis = 0, color = 'rgba(100,67,33,0.6)')


o_text = [ [o_data[i][0],str(i)] for i in range(len(o_data))]

H.add_data_set(o_text, 'series', 'A', yAxis = 0)


options = {'rangeSelector' : {'selected' : MIN_LEN},    
           'title': {'text' : i_data['ticker'].values[0]+' Stock Price'},
           'yAxis': {'tickAmount': 20, 'labels':{'y': 4, 'x':10} }, 
           'chart': {'height':600},
           'tooltip': {'pointFormat': '<br>Index: {point.index}</br><br>{series.name}</br><br/>O: {point.open}<br/>H: {point.high}<br/>L: {point.low}<br/>C: {point.close}'} }


H.set_dict_options(options)

IPython.display.HTML(H.htmlcontent)




In [465]:
# Export training data to o_data

index_breakout = [1369 ] # input numbers here
index_not_breakout_param = 10

index_not_breakout = [] 

for i in index_breakout:
    index_not_breakout = index_not_breakout + list(range(i-index_not_breakout_param, i)) + list(range(i+1,i+1+index_not_breakout_param))

t_data = np.empty((0,63*3+1))    

for i in index_not_breakout:
    t_1 = i_data.loc[i-63+1:i,['adj_high','adj_low','adj_close']]
    
    t_2 = np.hstack((t_1['adj_high'].values, t_1['adj_low'].values, t_1['adj_close'].values, 1))
    t_data = np.vstack((t_data, t_2))

for i in index_breakout:
    t_1 = i_data.loc[i-63+1:i,['adj_high','adj_low','adj_close']]
    
    t_2 = np.hstack((t_1['adj_high'].values, t_1['adj_low'].values, t_1['adj_close'].values, 0))
    t_data = np.vstack((t_data, t_2))
    
o_data63 = np.vstack((o_data63, t_data))
o_data63.shape

(1173, 190)

In [462]:
# Export completely "wrong" training data to o_data

index_not_breakout = [904]  # Enter index number here

index_not_breakout_param = 15

for i in index_not_breakout:
    index_not_breakout = index_not_breakout + list(range(i-index_not_breakout_param, i, 5)) + list(range(i+1,i+1+index_not_breakout_param, 5))

t_data = np.empty((0,63*3+1))    

for i in index_not_breakout:
    t_1 = i_data.loc[i-63+1:i,['adj_high','adj_low','adj_close']]
    
    t_2 = np.hstack((t_1['adj_high'].values, t_1['adj_low'].values, t_1['adj_close'].values, 1))
    t_data = np.vstack((t_data, t_2))

o_data63 = np.vstack((o_data63, t_data))
o_data63.shape

(1152, 190)

In [23]:
IPython.display.HTML(h)

In [22]:
h = '''
<!DOCTYPE html>\n
<html lang="en">\n    
	<head>\n        
		<meta charset="utf-8" />\n        
		<link href="https://www.highcharts.com/highslide/highslide.css" rel="stylesheet" />\n        
		<script type="text/javascript" src="https://ajax.googleapis.com/ajax/libs/jquery/1.9.1/jquery.min.js"></script>\n        
		<script type="text/javascript" src="https://code.highcharts.com/stock/highstock.js"></script>\n        
		<script type="text/javascript" src="https://code.highcharts.com/stock/modules/exporting.js"></script>\n 
        <script type="text/javascript" src="https://code.highcharts.com/stock/modules/drag-panes.js"></script>\n
		<script type="text/javascript" src="https://code.highcharts.com/highcharts-more.js"></script>\n    
	</head>\n    
	<body style="margin:0;padding:0">\n                
		<div id="container" style="height:600px;">Loading....</div>\n\n\n    
		<script>\n        
		$(function() {

    Highcharts.stockChart('container', {

        rangeSelector: {
            selected: 1
        },

        title: {
            text: 'AAPL Historical'
        },

        yAxis: [{
            labels: {
                align: 'right',
                x: -3
            },
            title: {
                text: 'OHLC'
            },
            height: '60%',
            lineWidth: 2,
            resize: {
                enabled: true
            }
        }, {
            labels: {
                align: 'right',
                x: -3
            },
            title: {
                text: 'Volume'
            },
            top: '65%',
            height: '35%',
            offset: 0,
            lineWidth: 2
        }],

        tooltip: {
            split: true
        },

        series: [{
            type: 'candlestick',
            name: 'AAPL',
            data: [[1557360000000.0, 37.366902800599206, 37.366902800599206, 37.27722223387775, 37.312098009824986, 0.0], [1557446400000.0, 37.45658336732064, 37.45658336732064, 37.28728638636539, 37.28728638636539, 0.0], [1557705600000.0, 35.97187176271016, 35.97187176271016, 35.96041257918465, 35.96041257918465, 0.0], [1557792000000.0, 36.31066501476892, 36.3650712252466, 36.31066501476892, 36.3650712252466, 0.0], [1557878400000.0, 35.88219119598872, 36.34095711730593, 35.88219119598872, 36.34095711730593, 0.0], [1557964800000.0, 36.58548612923307, 36.58548612923307, 36.44518586485109, 36.44518586485109, 0.0], [1558051200000.0, 36.450168118557826, 36.6394937594142, 36.2221801889371, 36.25077832521383, 0.0], [1558310400000.0, 36.14126838873955, 36.14126838873955, 36.14126838873955, 36.14126838873955, 0.0], [1558396800000.0, 36.10141035908557, 36.509257647519846, 36.03165880719112, 36.509257647519846, 0.0], [1558483200000.0, 36.66938728165467, 36.66938728165467, 36.16119740356653, 36.16119740356653, 0.0], [1558569600000.0, 35.71199740936625, 35.71199740936625, 35.71199740936625, 35.71199740936625, 0.0]]
        }, {
            type: 'column',
            name: 'Volume',
            data: [1,2,3,4,5,6,7,8,9,10,5],
            yAxis: 1
        }]
    });

});
		</script>\n\n        \n    
	</body>\n
</html>
'''